# Clase Extra 2 - Ensamblaje de Modelos

---

## Ensamblaje de Modelos

Es muy probable que para los problemas complejos, los clasificadores vistos anteriormente (KNN, Decision Trees, Bayes) no sean capaces de producir modelos lo suficientemente complejo como para resolverlos. 

Si bien el camino usual para resolver el problema de la complejidad es crear modelos de redes neuronales, estos por lo general requieren una gran cantidad de datos para generalizar correctamente. Por ende, si no se posee una mayor cantidad de datos, no siempre será la elección correcta.

Una opción interesante para tener en cuenta es el de utilizar **ensamblaje de modelos**. 

Los modelos generados por ensamblaje no son otra variedad de modelos más complejos y distintos a los vistos anteriormente, si no que son frameworks para generar **meta-modelos**, que no son otra cosa que combinaciones de muchos modelos poco complejos y con poca precisión (*weak-learners*).
La hipótesis principal de estos modelos es que la combinación de varios *weak-learners* poco precisos permite generar modelos mucho más complejos con la misma cantidad de datos.

Los *weak learners* pueden ser entrenados con pocos datos y muy rápidamente; tal como caso de los árboles de decisión. 
La combinación de estos modelos poco complejos se hace a través de algún mecanismo de agregación/votación. 



Existen dos paradigmas de creación de modelos ensamblados: **Bagging y Boosting**.

### Bagging (Bootstrap Aggregating)

**Bagging** consiste en entrenar varios *weak learners* (ej, trees) usando distintos muestreos del dataset de entrenamiento y luego combinar los modelos entrenados.


<br>

<div align='center'>
    <img src='./resources/ensemble_bagging.png' width=800/>
</div>

<br>
<div align='center'>
    Fuente: <a href='https://en.wikipedia.org/wiki/Bootstrap_aggregating'>Boosting en Wikipedia</a>
</div>
<br>



Un ejemplo clásico de este es `RandomForest`.
Su funcionaimento es relativamente sencillo: 

Supongamos que se entrenarán $B$ ($B$ de bags) árboles de decisión y que cada arbol será entrenado con $n$ datos. Entonces,



- Por cada $b=1,...B$:
    - Obtenemos un subconjunto de entrenamiento con n elementos a través de un muestreo con reemplazo (*bootstrap*) sobre conjunto de entrenamiento original.
    - Entrenamos un árbol de decisión para regresión o clasificación $f_b$ usando ese set.

Luego, dado un ejemplo por predecir $x$, para obtener los valores de:

- Una regresión se calcula: 

$$\hat{y} = \frac{1}{B}\sum_{b=1}^{B}f_b(x)$$

- Una clasificación, se retorna la clase con más votos. 


EL siguiente ejemplo consiste en un problema de detección de fraudes bancarios. 
Para mostrar el funcionamiento de Random Forest, evaluaremos la tarea anterior usando un modelo `DecisionTree` y compararemos el aumento de rendimiento usando luego un `RandomForest`

## Credit Card Fraud

El problema de esta clase consiste en, dados ciertos atributos preprocesados usando PCA (por lo que no sabemos que codifica cada dimensión), generar un clasificador que prediga si una fila representa o no un fraude bancario.

La fuente de los datos la pueden encontrar en [creditcardfraud en Kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud).

In [2]:
import pandas as pd

df = pd.read_pickle('./resources/creditcard.pickle')
df = df.drop(columns=['Time'])
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


#### Desablance de Clases

Una de las cosas que pueden notar de este dataset es un tremendo desbalance de clases: 284315 no fraudes vs 492 fraudes!

In [3]:
# normalise the amount column
features = df.drop(columns=['Class']) 
labels = df['Class']
# as you can see there are 492 fraud transactions.
labels.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    shuffle=True, 
                                                    stratify=labels,
                                                    test_size=0.3,
                                                    random_state=10)

#### Pipeline con DecisionTree

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix

# generar el pipeline
pipe1 = Pipeline([('Preprocesamiento',
                   ColumnTransformer([
                       ('Scaler', StandardScaler(), ['Amount']),
                   ],
                                     remainder='passthrough')),
                  ('Decision Tree', DecisionTreeClassifier(random_state=44))])

# entrenar
pipe1.fit(X_train, y_train.ravel())

# predecir
y_pred = pipe1.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Matriz de confusión

[[85256    39]
 [   41   107]]

Reporte de Clasificación

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.73      0.72      0.73       148

    accuracy                           1.00     85443
   macro avg       0.87      0.86      0.86     85443
weighted avg       1.00      1.00      1.00     85443



#### Pipeline Con RandomForest

In [8]:
# generar el pipeline
pipe2 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('Random Forest', RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=44))
    ])

# entrenar
pipe2.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe2.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Matriz de confusión

[[85288     7]
 [   31   117]]

Reporte de Clasificación

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.94      0.79      0.86       148

    accuracy                           1.00     85443
   macro avg       0.97      0.90      0.93     85443
weighted avg       1.00      1.00      1.00     85443



Como se puede ver, `RandomForest` incremento en la precisión y recall de la clase 1 con respecto al `DecisionTree`

### Boosting

El segundo enfoque para crear modelos ensamblados es Boosting. En este caso, cada modelo se entrena con todos los datos, pero los modelos se entrenan de forma iterativa de tal forma que los modelos siguientes intentan corregir los errores de los modelos anteriores.

<br>

<div align='center'>
    <img src='./resources/ensemble_boosting.png' width=800/>
</div>

<br>
<div align='center'>
    Fuente: <a href='https://en.wikipedia.org/wiki/Boosting_(machine_learning)'>Boosting en Wikipedia</a>
</div>
<br>

La idea, en términos simples, es que los primeros modelos predigan bien los casos más generales, mientras que los modelos más profundos se encarguen de predecir los casos más particulares y dificiles.



El modelo ensamblado final será una combinación de los modelos creados en el proceso iterativo. Uno de los algoritmos más populares para crear este tipo de modelos es `GradientBoosting` (usaremos `HistGradientBoostingClassifier`, una implementación mucho más rápida en el ejemplo). Una explicación de su idea principal viene a continuación.


Dada $M$ iteraciones, el algoritmo para generar un modelo para regresión es:

- Comenzamos con un modelo constante $f = f_0$
- Luego, por cada label $i=1,\dots,N$, la transformamos a $\hat{y}_i \leftarrow y_i - f(x_i)$. $\hat{y}_i$ es llamado residual y su conjunto se transforma en las nuevas labels de $x$.
- Ahora, entrenamos un nuevo modelo (tree) $f_1$ y redefinimos $f = f_0 + \alpha f_1$. $\alpha$ es conocido como learning rate.
- Computamos nuevamente los residuales y calculamos un nuevo modelo $f_2$. Luego, fijamos el modelo predictivo como $f = f_0 + \alpha f_1 + \alpha f_2$.
- Repetimos hasta que se cumplan $M$ iteraciones.


La idea general es que en cada iteración los residuales más cercanos a 0 son los que se están clasificando correctamente mientras que los residuales con valores más altos son mal predichos se espera que se vayan corrigiendo en los modelos más profundos hasta alcanzar el modelo $M$.

El caso de la clasificación es un poco más complejo y utiliza modelos de Regresión Logística para lograrlo. Queda propuesta su profundización.

#### Pipeline con GradientBoosting

In [9]:
from sklearn.ensemble import HistGradientBoostingClassifier

# generar el pipeline
pipe3 = Pipeline(
    [('Preprocesamiento', ColumnTransformer([
        ('Scaler', StandardScaler() , ['Amount']),
    ], remainder='passthrough')),
     ('HistGradientBoostingClassifier', HistGradientBoostingClassifier(
         random_state=44, 
         l2_regularization=10, 
         max_depth=15,
         max_iter=1000))
    ])

# entrenar
pipe3.fit(X_train, y_train.ravel())
 
# predecir
y_pred = pipe3.predict(X_test)

# reporte de clasificación
print('Matriz de confusión\n')
print(confusion_matrix(y_test, y_pred))
print('\nReporte de Clasificación\n')
print(classification_report(y_test, y_pred))

Matriz de confusión

[[85286     9]
 [   33   115]]

Reporte de Clasificación

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.93      0.78      0.85       148

    accuracy                           1.00     85443
   macro avg       0.96      0.89      0.92     85443
weighted avg       1.00      1.00      1.00     85443



Nuevamente vemos mejores resultados en este tipo de clasificador con respecto al árbol básico. 


En general, Gradient Boosting puede ser más poderoso que los basados en bagging, pero si entrenamiento de caracter secuencial puede hacelo mucho más lento.

**Referencias**


- [1] The Hundred-Page Machine Learning Book, Andriy Burkov. Artículo 7.5